In [1]:
import pandas as pd

In [2]:
csv = "data/1day_data.csv"
df = pd.read_csv(csv)
df = df.drop_duplicates()


In [3]:
alt_inaccuracy = 50 # погрешность в изменении высоты
# Функция для получения доли участков, где самолёт менял высоту
def get_altitude_rate(id, df):
    rows = df.loc[df['id'] == id]
    point_amount = len(rows)
    if point_amount <= 1:
        return 1.
    altitudes = rows['altitude']
    alt_delta_ratio = 0
    for i in range(1, point_amount):
        if abs(altitudes.iloc[i] - altitudes.iloc[i-1]) < alt_inaccuracy:
            alt_delta_ratio += 1
    return alt_delta_ratio / (point_amount - 1)

In [4]:
heading_inaccuracy = 2 # погрешность в изменении направления
# Функция для получения доли участков, где самолёт менял направление
def get_heading_rate(id, df):
    rows = df.loc[df['id'] == id]
    point_amount = len(rows)
    if point_amount <= 1:
        return 1.
    headings = rows['heading']
    head_delta_ratio = 0
    for i in range(1, point_amount):
        if abs(headings.iloc[i] - headings.iloc[i-1]) < heading_inaccuracy:
            head_delta_ratio += 1
    return head_delta_ratio / (point_amount - 1)

In [5]:
# Функция для получения доли участков, где самолёт менял высоту и направление
def get_heading_altitude_rate(id, df):
    return get_heading_rate(id, df) * get_altitude_rate(id, df)

In [6]:
ids = {id: df['id'].unique() for id in df['id']}
altitude_rates = []
heading_rates = []
heading_altitude_rates = []

# создаём новый датафрейм, группируя информацию по id полётов

for id in ids:
    altitude_rates.append(get_altitude_rate(id, df))
    heading_rates.append(get_heading_rate(id, df))
    heading_altitude_rates.append(get_heading_altitude_rate(id, df))
print(altitude_rates)
print(heading_rates)
print(heading_altitude_rates)

df = pd.DataFrame(list(zip(ids, altitude_rates, heading_rates, heading_altitude_rates)),
    columns = ['ids', 'altitude_rates', 'heading_rates', 'heading_altitude_rates'])
df = df[df['heading_altitude_rates'] != 0]
display(df)


[1.0, 0.6666666666666666, 1.0, 1.0, 0.0, 0.5, 0.22727272727272727, 1.0, 0.5, 1.0, 0.25, 1.0, 0.0, 0.0, 1.0, 1.0, 0.7058823529411765, 0.043478260869565216, 0.6666666666666666, 0.6666666666666666, 1.0, 0.8333333333333334, 0.2, 1.0, 1.0, 0.6666666666666666, 0.6666666666666666, 1.0, 0.16666666666666666, 1.0, 1.0, 1.0, 0.8888888888888888, 0.3142857142857143, 0.0, 1.0, 0.3333333333333333, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8888888888888888, 0.0, 1.0, 0.4444444444444444, 0.0, 1.0, 1.0, 1.0, 0.5333333333333333, 1.0, 1.0, 1.0, 1.0, 1.0, 0.6, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.6111111111111112, 1.0, 1.0, 1.0, 0.5714285714285714, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.4, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5454545454545454, 1.0, 0.9, 1.0, 1.0, 1.0, 1.0, 0.5714285714285714, 1.0, 0.375, 1.0, 1.0, 1.0, 1.0, 0.07142857142857142, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.6153846153846154, 1.0, 1.0, 1.0, 0.3333333333333333, 1.0, 1.0, 1.0, 1.0, 1

,ids,altitude_rates,heading_rates,heading_altitude_rates
0,3265ef91,1.000000,0.600000,0.600000
1,3265cea8,0.666667,0.733333,0.488889
2,3265ac16,1.000000,1.000000,1.000000
5,32658ba9,0.500000,0.875000,0.437500
6,32654117,0.227273,0.681818,0.154959
...,...,...,...,...
892,3268d39c,1.000000,1.000000,1.000000
894,32695334,0.909091,0.363636,0.330579
895,3269593e,0.142857,1.000000,0.142857
896,32699897,1.000000,1.000000,1.000000


In [7]:
print("Описание доли маршрута полёта без изменения высоты")
print(df['altitude_rates'].describe())
print("Описание доли маршрута полёта без поворота")
print(df['heading_rates'].describe())
print("Описание доли маршрута полёта без поворота и без снижения высоты")
print(df['heading_altitude_rates'].describe())


Описание доли маршрута полёта без изменения высоты
count    752.000000
mean       0.766797
std        0.322179
min        0.025641
25%        0.500000
50%        1.000000
75%        1.000000
max        1.000000
Name: altitude_rates, dtype: float64
Описание доли маршрута полёта без поворота
count    752.000000
mean       0.692948
std        0.284304
min        0.055556
25%        0.500000
50%        0.714286
75%        1.000000
max        1.000000
Name: heading_rates, dtype: float64
Описание доли маршрута полёта без поворота и без снижения высоты
count    752.000000
mean       0.537552
std        0.354150
min        0.008889
25%        0.214048
50%        0.498270
75%        1.000000
max        1.000000
Name: heading_altitude_rates, dtype: float64
